### Load libraries & config

In [1]:
from datetime import date, datetime
import girder_client as gc
import json
import os
from urllib.parse import quote
import random
import re
import string
import tzlocal
from urllib.request import urlopen

with open("config.json", "r") as fp:
    config=json.load(fp)

### Connect to Girder

In [2]:
girder_connection = gc.GirderClient(
    apiUrl="{}/api/v1/".format(
        config["girder-production"]["host"]
    )
)
girder_connection.authenticate(
    username=config["girder-production"]["user"],
    password=config["girder-production"]["password"]
)

In [4]:
print('["' + '", "'.join([u['_id'] for u in girder_connection.get('user')]) + '"]')

["5c4789c145e8fc71d167bd0a", "5c92a7868990750e252b5f72", "5c65d3b37c7e6d0e0e770685", "5b634cc00fc13a0df633ae02", "5b634cbe0fc13a0df633adfe", "5c1411e0c25b5a144b269e63", "5c82e488400fe179574070a3", "5bd08898336da80de91457a5", "5c5354024e9a670deb99b6d7", "5c76cb557c7e6d4670933620", "5c6ae4f67c7e6d1ed94ac4b6", "5c9105988990750e252b5f2e", "5c58f8ba4e9a670deb99b9c7", "5c632c872a12c10e79249b58", "5c1c29e4c25b5a6a94d5fc13", "5c34cf14c25b5a625531cc34", "5c34cba7c25b5a625531cc30", "5c642f992a12c10e79249c1c", "5c7851dc7c7e6d4670933699", "5b7f1d7ae534fe32c5f5db8b", "5c547c5c4e9a670deb99b8b4", "5c2e8ec3c25b5a6a94d60097", "5c65e5297c7e6d0e0e77068b", "5c8ab350400fe17cd16be9f8", "5ca37c851535460e001811a2", "5bf825c1336da85b0dfed57d", "5c36692da0e4760df6a03426", "5c93c2588990750e252b5f97", "5bd08d18336da80de91457eb", "5c6325222a12c10e79249b44", "5c65e72f7c7e6d1ed94ac407", "5c62ad242a12c10e79249abd", "5c53f4844e9a670deb99b76c", "5c2fcf8bc25b5a625531c8fa", "5c9a42ef0419cc2016a7d2c7", "5c58a87c4e9a670deb

In [ ]:
activity_sets = girder_connection.get(
    "folder?parentType=collection&parentId={}".format(
        girder_connection.get(
            "collection?name=Volumes"
        )[0]["_id"]
    )
)
activity_sets = [a for a in activity_sets if 'meta' in a]

### Authorize Viewers to View their Users as Authorized in Admin Panel

In [ ]:
for a in [a for a in activity_sets if 'meta' in a]:
    print("Activity Set: {}".format(a['name']))
    for v in a['meta']['members']['viewers']:
        print("Viewer: {}".format(girder_connection.getUser(v)['firstName']))
        for u in a['meta']['members']['viewers'][v]:
            try: 
                access = girder_connection.get("user/{}/access".format(u))
            except:
                access = {'users': []}
            print("User: {}".format(girder_connection.getUser(u)['firstName']))
            print("access: {}".format(", ".join([user['name'] for user in access['users']])))
            if (v not in [user['id'] for user in access['users']]):
                access['users'].append({'id': v, 'level': 0})
                girder_connection.put("user/{}/access".format(u),
                    data={"access":json.dumps(access)}
                )
            try:
                responses = girder_connection.get(
                    "folder?parentType=user&parentId={}&name=Responses".format(
                        u
                    )
                )
                for responses_folder in responses:
                    print("Responses: {}".format(responses_folder['name'] if 'name' in responses_folder else responses_folder))
                    try:
                        access = girder_connection.get(
                            "folder/{}/access".format(
                                responses_folder['_id']
                            )
                        )
                        if v not in [uid["id"] for uid in access['users']]:
                            access['users'].append({'id': v, 'level': 0})
                        girder_connection.put(
                            'folder/{}/access'.format(
                                responses_folder['_id']
                            ),
                            data={
                                "access":json.dumps(access),
                                "recurse":False
                            },
                        )
                        print(access)
                        activity_set_responses = girder_connection.get(
                            "folder?parentType=folder&parentId={}&name={}".format(
                                responses_folder['_id'],
                                a['name']
                            )
                        )
                        for activity_set_response_folder in activity_set_responses: 
                            print("Responses Folder: {}".format(activity_set_response_folder['name']))
                            access = girder_connection.get(
                                "folder/{}/access".format(
                                    activity_set_response_folder['_id']
                                )
                            )
                            print("access: {}".format(", ".join([uid["name"] for uid in access['users']])))
                            if v not in [uid["id"] for uid in access['users']]:
                                access['users'].append({'id': v, 'level': 0})
                            girder_connection.get('folder/{}'.format(activity_set_response_folder['_id']))['name']
                            girder_connection.put(
                                'folder/{}/access'.format(
                                    activity_set_response_folder['_id']
                                ),
                                data={
                                    "access":json.dumps(access),
                                    "recurse":True
                                },
                            )
                    except:
                        activity_set_responses = None
            except:
                responses = None
        print("\n")

### Authorize Managers to View Users as Authorized in Admin Panel

In [ ]:
for a in [a for a in activity_sets if 'meta' in a]:
    for u in a['meta']['members']['users']:
        try:
            access = girder_connection.get(
                'user/{}/access'.format(u)
            )
            for m in a['meta']['members']['managers']:
                if m not in [uid['id'] for uid in access['users']]:
                    print(m)
                    access['users'].append({'id': m, 'level': 0})
            print(access)
            girder_connection.put(
                'user/{}/access'.format(u),
                data={"access":json.dumps(access)}
            )
        except:
            access = {}

### Authorize Healthy Brain Network Data Access Group Data Access to All **EMA: Parent** Activity Data

In [ ]:
for user in girder_connection.getFolder('5bd87caa336da80de9145af2')['meta']['members']['users']:
    try:
        responses = girder_connection.get(
            "folder?parentType=user&parentId={}&name=Responses".format(
                user
            )
        )
        for responses_folder in responses:
            try:
                ema_parent_responses = girder_connection.get(
                    "folder?parentType=folder&parentId={}&name=EMA%3A%20Parent".format(
                        responses_folder['_id']
                    )
                )
                for ema_parent_response_folder in ema_parent_responses: 
                    access = girder_connection.get(
                        "folder/{}/access".format(
                            ema_parent_response_folder['_id']
                        )
                    )
                    if "5bab8fc80a76ac049becd5ec" not in [gid["id"] for gid in access['groups']]:
                        access['groups'].append({'id': '5bab8fc80a76ac049becd5ec', 'level': 0})
                print(access)
                girder_connection.put(
                    'folder/{}/access'.format(
                        ema_parent_response_folder['_id']
                    ),
                    data={
                        "access":json.dumps(access),
                        "recurse":True
                    },
                )
            except:
                ema_parent_responses = None
    except:
        responses = None

### Consolidate duplicated permissions

In [ ]:
for u in girder_connection.get('user'):
    access = girder_connection.get(
        'user/{}/access'.format(u['_id'])
    )
    new_access = {'groups': access['groups'], 'users': []}
    for au in access['users']:
        if au['id'] not in [uid['id'] for uid in new_access['users']]:
            new_access['users'].append(au)
    print(new_access)
    girder_connection.put(
        'user/{}/access'.format(u["_id"]),
        data={"access":json.dumps(new_access)}
    )

### Purge non-existent users from admin panel

In [ ]:
all_assignees = list({
    *{
        e for a in
        activity_sets for e in
        a['meta']['members']['editors']
    },
    *{
        m for a in
        activity_sets for m in
        a['meta']['members']['managers']
    },
    *{
        u for a in
        activity_sets for u in
        a['meta']['members']['users']
    },
    *{
        v for a in
        activity_sets for v in
        a['meta']['members']['viewers']
    },
    *{
        u for a in
        activity_sets for v in
        a['meta']['members']['viewers'] for u in
        a['meta']['members']['viewers'][v] 
    }
})
purge_list = []
for assignee in all_assignees:
    try:
        girder_connection.getUser(assignee)
    except:
        purge_list.append(assignee)
print(purge_list)
for a in activity_sets:
    original_members = a['meta']['members'].copy()
    for role in ['editors', 'users', 'managers']:
        a['meta']['members'][role] = list(set(a['meta']['members'][role]) - set(purge_list))
    for purgee in purge_list:
        a['meta']['members']['viewers'].pop(purgee, None)
    for v in a['meta']['members']['viewers']:
        a['meta']['members']['viewers'][v] = list(set(a['meta']['members']['viewers'][v]) - set(purge_list))
    if(original_members != a['meta']['members']):
        girder_connection.put(
            'folder/{}'.format(
                a['_id']
            ),
            data={
                'metadata':json.dumps(
                    a['meta']
                )
            }
        )

In [ ]:
for manager in [a for a in activity_sets if a['name'] == "EMA: Parent"][0]['meta']['members']['managers']:
    u = girder_connection.getUser(manager)
    print("{} {}".format(u['firstName'], u['lastName']))

In [ ]:
all_assignees = list({
    *{
        e for a in
        activity_sets for e in
        a['meta']['members']['editors']
    },
    *{
        m for a in
        activity_sets for m in
        a['meta']['members']['managers']
    },
    *{
        u for a in
        activity_sets for u in
        a['meta']['members']['users']
    },
    *{
        v for a in
        activity_sets for v in
        a['meta']['members']['viewers']
    },
    *{
        u for a in
        activity_sets for v in
        a['meta']['members']['viewers'] for u in
        a['meta']['members']['viewers'][v] 
    }
})
for assignee in all_assignees:
    if assignee not in [a for a in activity_sets if a['name'] == "EMA: Parent"][0]['meta']['members']['users']:
        u = girder_connection.getUser(assignee)
        print("{} {} {}".format(u['firstName'], u['lastName'], u['_id']))